In [ ]:
from collections import Counter
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
TRAIN_TEXT_FILE_PATH = 'allsoch.txt'

with open(TRAIN_TEXT_FILE_PATH, encoding = 'utf-8') as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'и', 'т', 'н', 'с', 'в', 'р', 'л', 'м', 'к', 'д', 'у', 'п', ',', 'я', 'ь', 'ы', 'г', 'ч', 'з', 'б', '.', 'й', '\n', 'ж', 'ю', 'х', 'ш', 'ц', 'щ', '«', '»', 'В', 'э', 'А', 'С', 'П', 'Н', '-', ':', 'И', 'Т', 'О', 'ф', 'Д', 'К', 'М', '–', 'Л', '!', 'Я', 'Б', '…', 'ё', 'Г', 'Р', 'Е', '"', 'Ч', '?', 'Э', 'З', 'Ю', 'Ф', 'У', '1', '0', 'Ж', 'ъ', '(', ')', 'Ш', '“', '”', '2', '9', '—', 'Х', 'Ц', '8', '3', ';', '%', '5', '6', 't', '4', 'I', 'u', 's', 'R', 'i', 'Щ', '№', 'r', 'N', 'P', 'y', 'o', 'T', 'w', 'e']


In [ ]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [ ]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [ ]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=256, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 3.277037124633789
 но ре  тет  тоноснети несте  с  тово   рот р п ст  т к ре он тачо ток  вотост ст   нет те с т по вото т кат  тодапоп воп петоте   ет мос  я та но с т    ск   о  тено мо т м  тет   ни тот но  ве н т  н
Loss: 2.909836139678955
 позосто в в вет в рони ве поса свизой на ном во в пога в сростом в пасто зезе ва стате данов ва на това пос повасто в в вови равореровон на коском ва ве свесто сто та в на посто тово сто стаз не белос
Loss: 2.709603304862976
                                                   
 поне в толема сто пороловалелим чолителати в в пореденним толелисто стово тра е в стовонна продосто тостова ромим дестоворим повеме ном стово рате в
Loss: 2.540844645500183
 
 
          
 
 
 
 
 
    
 
 
 
 
 
    
 
    
 
 
 
 
 
   
 
 
 
 
 
 
 
 
 
 
 
 
 
    
 
 
 
 
 
 
 
                          
 
 
 
 
 
       
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Loss: 2.3944212055206298
 в совлика ором пронно не в вот отвертет в сторов в вот отства в люденн

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), 'model.pt')

with open('char_to_idx.pickle', 'wb') as f:
  pickle.dump(char_to_idx, f)

with open('idx_to_char.pickle', 'wb') as f:
  pickle.dump(idx_to_char, f)

In [ ]:
model.eval()
print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='. '
    )
)

. Вот постоянно таких произведения автор с гордостью повествует о том, что переполнены отличается о встрече по мнению автора, ответственности за свои современники на земле.
 
 Я полностью согласна с мнением автора. Действительно, молчание и приняли предназначение к прошлого человека. Он считает, что не зарода в тексте с умастер с ним. По мнению автора, не может произведении Н. Татаринцев «Горе относится Борису Карам отречении происходит и высоко простого солдата и очень подвиг не было на стало тому, что перед нами люди века, а когда все мелимера стало судьбе приводит к душевной добротой, что не боролой этого является перед нами люди в своей душе перед ему проявляется героиня романа Л.Н. Толстого «Война и мир» Л. Толстого, вера в той стремления к произведении «страдания, которые не повлиял на войне самоубийство в нашей жизни. 
 
 Я полностью разделяю точку зрения автора, осуждает нас подвиг нашего человека открыть свой поступки и общественный вопрос, поднимаемый автором, которые потребн